In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from time import time
import numpy as np
import plotly.graph_objects as go
import numba as nb
from numba import jit
import pickle

from DEFAULTS import PARENT_PATH
import ap_field_jit as ap
import pan_mod_jit as pm
import tele_geo_jit as tg

tele_geo_t = tg.initialize_telescope_geometry()

tg_th_1 =  tele_geo_t.th_1
tg_th2 = tele_geo_t.th2
tg_z_ap = tele_geo_t.z_ap
tg_th_fwhp = tele_geo_t.th_fwhp
tg_F_2 = tele_geo_t.F_2

tg_x_rotc = tele_geo_t.x_rotc
tg_y_rotc = tele_geo_t.y_rotc
tg_z_rotc = tele_geo_t.z_rotc

freq = 120.0 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01
k = 2 * np.pi / wavelength

dist = np.inf # set it infinitly far away
P_source = np.array([0, -7.2, dist])  # position of the source[m] 

tg_el0 = np.arctan(-(P_source[1] + 7.2) / P_source[2]) # elevation of telescope based on position of source tower [rad]
tg_az0 = np.arctan(-P_source[0] / P_source[2]) # azimuth of telescope based on position of source tower [rad]

error = 0 # mean mirror surface error [um]
adj1 = np.random.randn(1092) * error
adj2 = np.random.randn(1092) * error
save = 0
pan_mod2 = pm.panel_model_from_adjuster_offsets(
    2, adj2, 1, save
)  # Panel Model on M2
pan_mod1 = pm.panel_model_from_adjuster_offsets(
    1, adj1, 1, save
)  # Panel Model on M1

N_scan = 64
theta = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, N_scan)
phi = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, N_scan)

focus = np.array([0,0,0]) #the focus point

In [2]:
path = f"{PARENT_PATH}/data/tung/JIT_test_sleep"  # path where you save far-field simulations.
if not os.path.exists(path):
    os.makedirs(path)

fp_x_N = 1
fp_z_N = 1
range_factor = 80
fp_x = np.linspace(focus[0]-range_factor*wavelength, focus[0]+range_factor*wavelength, 200) # unit: [m]
fp_z = np.linspace(focus[2]-range_factor*wavelength, focus[2]+range_factor*wavelength, 200)  # unit: [m]

N_scan = 64
theta = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, N_scan)
phi = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, N_scan)

def save_parameter():
    save_parameters = dict(
                        wavelength=wavelength, 
                        P_source=P_source, 
                        surface_error=error,
                        N_scan=N_scan, 
                        fp_x_N=fp_x_N, 
                        fp_z_N=fp_z_N,
                        fp_x_range=(fp_x[0], fp_x[-1]),
                        fp_z_range=(fp_z[0], fp_z[-1])
                        )
    filename = "parameters"
    with open(f"{path}/{filename}.pys", "wb") as file:
        pickle.dump(save_parameters, file)
save_parameter()

sim_size = fp_x_N*fp_z_N
savearray = np.zeros((sim_size, 17, N_scan*N_scan))
@jit(nopython=True, parallel=True, fastmath=True)
def simulation(fp_x_N, fp_z_N):
    savearray = np.zeros((sim_size, 17, N_scan*N_scan))
    for ii in nb.prange(fp_x_N):
        for jj in nb.prange(fp_z_N):
            P_rx = np.array([fp_x[ii], focus[2], fp_z[ii]])
            rxmirror_t = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
            apfield_t = ap.aperature_fields_from_panel_model(pan_mod1, pan_mod2, \
                                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                                    tg_F_2, theta, phi, rxmirror_t
                                                    )
            savearray[ii*fp_x_N+jj,:,:] = apfield_t
            # print("Progress:", "(",  (ii*fp_x_N+jj+1) * 100.0 / sim_size, '% already done)')
simulation(fp_x_N, fp_z_N)
filename = "apfield_sim"
with open(f"{path}/{filename}.pys", "wb") as file:
    pickle.dump(savearray, file)